In [11]:
import sys, os
import numpy as np
import glob
import pandas as pd
import math

from scipy import stats
import matplotlib.pyplot as plt
from astropy.io import fits

from bokeh.io import output_notebook, show, export_png,export_svgs, save
from bokeh.plotting import figure, show, output_file
import bokeh.palettes
from bokeh.layouts import column,gridplot
from bokeh.models import HoverTool, ColumnDataSource, LinearColorMapper
from bokeh.models.glyphs import Text
from bokeh.palettes import Category20
from bokeh.transform import linear_cmap

from ics.cobraCharmer import pfi as pfiControl
from ics.cobraCharmer import pfiDesign
from astropy.table import Table



In [25]:
def plotJ1OntimeSpeed(GroupIdx, dataFrame, xrange, yrange):

    mapper = Category20[len(GroupIdx)]
    TOOLS = ['pan','box_zoom','wheel_zoom', 'save' ,'reset','hover']

    p = figure( tools=TOOLS, x_range=xrange, y_range=yrange,plot_height=500, plot_width=1000)

    colorcode = 0
    for i in GroupIdx:
        legendname="Fiber "+str(i)

        p.line(x=dataFrame['J1onTime'][dataFrame['fiberNo'] == i], y=dataFrame['J1_fwd'][dataFrame['fiberNo'] == i],\
        color=mapper[colorcode],line_width=2,legend=legendname)
        p.circle(x=dataFrame['J1onTime'][dataFrame['fiberNo'] == i], y=dataFrame['J1_fwd'][dataFrame['fiberNo'] == i],radius=0.1,\
            color=mapper[colorcode],fill_color=None)
        p.line(x=dataFrame['J1onTime'][dataFrame['fiberNo'] == i], y=dataFrame['J1_rev'][dataFrame['fiberNo'] == i],color=mapper[colorcode],line_width=2)
        p.circle(x=dataFrame['J1onTime'][dataFrame['fiberNo'] == i], y=dataFrame['J1_rev'][dataFrame['fiberNo'] == i],radius=0.1,\
            color=mapper[colorcode],fill_color=None)

        colorcode = colorcode + 1

    return p




class ontimeModel():
    
    def getThetaFwdSlope(self, pid, dataframe):

        ndf = dataframe.loc[dataframe['fiberNo'] == pid]

        onTimeArray = ndf['J1onTime'].values
        angSpdArray = ndf['J1_fwd'].values

        slope, intercept, r_value, p_value, std_err = stats.linregress(onTimeArray,angSpdArray)

        # If the slope is nan, that means the linear regression is failed.  Return zero instead of nan.
        if math.isnan(slope) is True:
            slope = 0

        return slope,intercept

    def getThetaRevSlope(self, pid, dataframe):

        ndf = dataframe.loc[dataframe['fiberNo'] == pid]

        onTimeArray = ndf['J1onTime'].values
        angSpdArray = ndf['J1_rev'].values

        slope, intercept, r_value, p_value, std_err = stats.linregress(onTimeArray,angSpdArray)

        # If the slope is nan, that means the linear regression is failed.  Return zero instead of nan.
        if math.isnan(slope) is True:
            slope = 0

        return slope,intercept

    def buildThetaModelfromDataFrame(self, dataframe, visibleFibers=False):

        # Reading all model and build a model list

        j1fwd_slope = []
        j1fwd_int = []
        
        j1rev_slope = []
        j1rev_int = []


        for pid in range(1,58):
            fw_s, fw_i=self.getThetaFwdSlope(pid,dataframe)
            rv_s, rv_i=self.getThetaRevSlope(pid,dataframe)
            
            j1fwd_slope.append(fw_s)
            j1fwd_int.append(fw_i)
            
            j1rev_slope.append(rv_s)
            j1rev_int.append(rv_i)
       
        self.j1fwd_slope = j1fwd_slope
        self.j1fwd_int   = j1fwd_int
        self.j1rev_slope = j1rev_slope
        self.j1rev_int   = j1rev_int




In [7]:
path='/Volumes/GoogleDrive/My Drive/PFS/CobraData/20190618/20190618/'
brokens = []
visibles= [e for e in range(1,58) if e not in brokens]
goodIdx = np.array(visibles) - 1
goodGroupIdx = {}
for group in range(6):
    goodGroupIdx[group] = goodIdx[goodIdx%6==group] + 1

In [9]:
dataarray=[]
pidarray=[]
otarray=[]
j1fwarray=[]
j1rvarray=[]
tarray=[20,30,40,50]

for tms in tarray:
    #print(tms)
    fw_file=path+f'{tms:0>4d}'+'/thetaSpeedFW.npy'
    rv_file=path+f'{tms:0>4d}'+'/thetaSpeedRv.npy'
    J1_fwd=np.load(fw_file)*180.0/math.pi 
    J1_rev=-np.load(rv_file)*180.0/math.pi 
    
    J1onTime=np.zeros(len(J1_rev))+tms
    pid=np.array(visibles)
    
    pidarray.append(pid)
    otarray.append(J1onTime)
    j1fwarray.append(J1_fwd)
    j1rvarray.append(J1_rev)
    
d={'fiberNo': np.array(pidarray).flatten(),'J1onTime':  np.array(otarray).flatten(), 
    'J1_fwd': np.array(j1fwarray).flatten(), 'J1_rev': np.array(j1rvarray).flatten()}
df = pd.DataFrame(d)



In [14]:
otm = ontimeModel()
otm.buildThetaModelfromDataFrame(df)

fwd_target = np.zeros(len(otm.j1fwd_int))+0.05
rev_target = np.zeros(len(otm.j1fwd_int))-0.05
newOntimeFwd1 = (fwd_target-otm.j1fwd_int)/otm.j1fwd_slope
newOntimeRev1 = (rev_target-otm.j1rev_int)/otm.j1rev_slope




In [15]:
newOntimeFwd1

array([41.9148294 , 36.68360766, 64.00481749, 38.27860735, 44.05894255,
       30.42345785, 31.4194849 , 33.22047399, 29.55107867, 41.95336435,
       33.67701723, 34.40503744, 34.56180175, 41.10234567, 25.27262047,
       47.47420493, 30.61250878, 31.86978797, 39.51761945, 41.01258955,
       26.7732313 , 40.25419218, 25.67111657, 44.14626428, 33.61583706,
       33.09488528, 27.98669252, 33.76363284, 35.7311819 , 33.06374346,
       38.46586822, 30.91981478, 43.78080198, 30.19366624, 42.85396378,
       33.10732205, 52.81029411, 37.52837812, 33.20738641, 36.90076825,
       36.54625727, 39.51024484, 32.88130821, 32.45648235, 54.73221465,
       28.67025721, 31.06453226, 28.3425794 , 77.1165331 , 39.89150558,
       34.81311448, 43.11689081, 31.183365  , 31.44972854, 35.7104142 ,
       44.30172351, 25.365056  ])

In [27]:
TOOLS = ['pan','box_zoom','wheel_zoom', 'save' ,'reset','hover']

thetarange = tarray

p1 = plotJ1OntimeSpeed(goodGroupIdx[0], df, [thetarange[0], thetarange[-1]], [-0.5,0.5])
p2 = plotJ1OntimeSpeed(goodGroupIdx[1], df, [thetarange[0], thetarange[-1]], [-0.5,0.5])
p3 = plotJ1OntimeSpeed(goodGroupIdx[2], df, [thetarange[0], thetarange[-1]], [-0.5,0.5])
p4 = plotJ1OntimeSpeed(goodGroupIdx[3], df, [thetarange[0], thetarange[-1]], [-0.5,0.5])
p5 = plotJ1OntimeSpeed(goodGroupIdx[4], df, [thetarange[0], thetarange[-1]], [-0.5,0.5])
p6 = plotJ1OntimeSpeed(goodGroupIdx[5], df, [thetarange[0], thetarange[-1]], [-0.5,0.5])

x = np.array([])
y1 = np.array([])
y2 = np.array([])
for tms in thetarange:
    #print(np.mean(df['J2_fwd'][df['onTime']==tms].values))
    x=np.append(x,tms)
    y1=np.append(y1,np.median(df['J1_fwd'][df['J1onTime']==tms].values))
    y2=np.append(y2,np.median(df['J1_rev'][df['J1onTime']==tms].values))

slope, intercept, r_value, p_value, std_err = stats.linregress(x[:8],y1[:8])
#fit=np.polyfit(x, y*100, 1)
#print(fit)

q = figure(tools=TOOLS, x_range=[thetarange[0]-10, thetarange[-1]+10], y_range=[-0.5,0.5],plot_height=500, plot_width=1000)
q.circle(x=df['J1onTime'], y=df['J1_fwd'],radius=0.3,\
        color='red',fill_color=None)
q.circle(x=x,y=y1, radius=0.5, color='blue')
legendtext=f'Y={slope:.8f}X{intercept:.2f}'
q.line(x=x[:8], y=x[:8]*slope+intercept,color='blue',line_width=3, legend=legendtext)

slope, intercept, r_value, p_value, std_err = stats.linregress(x[:8],y2[:8])
#fit=np.polyfit(x, y*100, 1)
q.circle(x=df['J1onTime'], y=df['J1_rev'],radius=0.3,\
        color='green',fill_color=None)
q.circle(x=x,y=y2, radius=0.5, color='#3B0F6F')
legendtext=f'Y={slope:.8f}X+{intercept:.2f}'
q.line(x=x[:8], y=x[:8]*slope+intercept,color='#3B0F6F',line_width=3, legend=legendtext)

output_file("theta.html")
save(column(p1,p2,p3,p4,p5,p6,q), filename="theta.html", \
    title='Theta On-time')



'/Users/chyan/Documents/workspace/ics_cobraCharmer/notebooks/theta.html'

In [31]:

initXML = '/Volumes/GoogleDrive/My Drive/PFS/CobraData/20190610/spare2_test.xml'
newXML = '/Volumes/Disk/Data/xml/spare2_opttheta_20190621.xml'
model = pfiDesign.PFIDesign(initXML)

thetaTable = 'thetaOntime.csv'

t=Table([pid, model.motorOntimeFwd1, otm.j1fwd_int, otm.j1fwd_slope, newOntimeFwd1,
         model.motorOntimeRev1,otm.j1rev_int, otm.j1rev_slope, newOntimeRev1],
         names=('Fiber No','Ori Fwd OT', 'FWD int', 'FWD slope', 'New Fwd OT',
                'Ori Rev OT', 'REV int', 'REV slope', 'New Rev OT'),
         dtype=('i2','f4', 'f4', 'f4','f4', 'f4', 'f4', 'f4', 'f4'))
t.write(thetaTable,format='ascii.ecsv',overwrite=True, 
        formats={'Fiber No':'%i','Ori Fwd OT': '%10.5f', 'FWD int': '%10.5f', 'FWD slope': '%10.5f', 'New Fwd OT': '%10.5f',\
        'Ori Rev OT': '%10.5f', 'REV int': '%10.5f', 'REV slope': '%10.5f', 'New Rev OT': '%10.5f'})

model.updateOntimes(thtFwd=newOntimeFwd1/1000.0, thtRev=newOntimeRev1/1000.0, fast=False)
model.updateOntimes(thtFwd=4.0*(newOntimeFwd1/1000.0), thtRev=4.0*(newOntimeRev1/1000.0), fast=True)

model.createCalibrationFile(newXML)

In [43]:
def plotBestThetaOnTimeFwd(DataFrame, OntimeModel, fiberNo):
    TOOLS = ['pan','box_zoom','wheel_zoom', 'save' ,'reset','hover']

    title_string=f"Fiber {fiberNo} Theta Fwd"
    p = figure(tools=TOOLS, x_range=[0,60], y_range=[0,0.4],
               plot_height=400, plot_width=500,title=title_string)
    p.xaxis.axis_label = 'On Time'
    p.yaxis.axis_label = 'Speed'
    dd=df.loc[df['fiberNo'] == fiberNo]

    newOntimeFwd1 = (0.05-otm.j1fwd_int[fiberNo-1])/otm.j1fwd_slope[fiberNo-1]


    x = range(10,50)
    y_predicted = [otm.j1fwd_slope[fiberNo-1]*i + otm.j1fwd_int[fiberNo-1]  for i in x]
    p.circle(x=newOntimeFwd1,y=[0.05],color='red', fill_color=None, radius=1.0)
    #p.circle(x=calibModel.motorOntimeSlowFwd2[fiberNo-1]*1000.0,y=speed_fwd[fiberNo-1]*180/math.pi,
    #color='blue', fill_color=None, radius=1.0)

    p.circle(x=dd['J1onTime'],y=dd['J1_fwd'])
    p.line(x,y_predicted,color='red')
    
    return p

def plotBestThetaOnTimeRev(DataFrame, OntimeModel, fiberNo):
    TOOLS = ['pan','box_zoom','wheel_zoom', 'save' ,'reset','hover']

    title_string=f"Fiber {fiberNo} Theta Rev"
    p = figure(tools=TOOLS, x_range=[0,60], y_range=[-0.4,0],
               plot_height=400, plot_width=500,title=title_string)
    p.xaxis.axis_label = 'On Time'
    p.yaxis.axis_label = 'Speed'
    dd=df.loc[df['fiberNo'] == fiberNo]

    newOntimeRev1 = (-0.05-otm.j1rev_int[fiberNo-1])/otm.j1rev_slope[fiberNo-1]


    x = range(10,50)
    y_predicted = [otm.j1rev_slope[fiberNo-1]*i + otm.j1rev_int[fiberNo-1]  for i in x]
    p.circle(x=newOntimeRev1,y=[-0.05],color='red', fill_color=None, radius=1.0)
    #p.circle(x=calibModel.motorOntimeSlowRev1[fiberNo-1]*1000.0,y=-speed_rev[fiberNo-1]*180/math.pi,
    #         color='blue', fill_color=None, radius=1.0)

    p.circle(x=dd['J1onTime'],y=dd['J1_rev'])
    p.line(x,y_predicted,color='red')
    
    return p



In [51]:
parray=[]

for f in goodIdx:
    p=plotBestThetaOnTimeFwd(df, otm,  f+1)
    parray.append(p)

grid = gridplot(parray, ncols=3, plot_width=400, plot_height=400)
show(grid)

In [50]:
parray=[]

for f in goodIdx:
    p=plotBestThetaOnTimeRev(df, otm,  f+1)
    parray.append(p)

grid = gridplot(parray, ncols=3, plot_width=400, plot_height=400)
show(grid)